# Unit 4 Extending Agent Capabilities with Built-In and Custom Tools

## Adding a Built-In Tool to Your Agent

## Exploring Agent Reasoning Steps

## Build a Custom Calculation Tool

## Exploring Tool Metadata in Agents

## Combining Tools for Smarter Agents